# Model Training Notebook

In [1]:
%matplotlib inline
import sys
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import numpy as np
from datetime import datetime
from IPython.core.display import display, HTML
import cv2
from PIL import Image
from pathlib import Path
import torchvision
import torch
import torch.nn as nn
import torch.nn.functional as F
import ast

from sklearn.model_selection import train_test_split

# repeatable functions housed in the utils file and imported here
from utils import *
from model_training_utils import train as training_loop
from model_training_utils import validate as validation_loop
from model_training_utils import hp_grid_search
from models import CNN, YOLO, VGG, PreTrainedVGG

## Create Training and Validation Datasets

In [2]:
df= pd.read_csv('./datasets/annotations_map.csv', converters={'new_bb': from_np_array})

In [3]:
df_train = df.reset_index()
X = df_train[['new_path','new_bb']]
Y = df_train['class']
X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.25, random_state=42)
print('Number of training observations: ', X_train.shape[0])
print('Number of validation observations: ', X_val.shape[0])

Number of training observations:  32
Number of validation observations:  11


## Build VGG
Implementation of VGG-16 architecture based on https://neurohive.io/en/popular-networks/vgg16/. VGG-16 is known for its high accuracy and speen on object detection tasks, largely attributed to its 3x3 kernel size.

vgg = VGG()

## Pre-trained VGG

In [4]:
preTrainedVGG = PreTrainedVGG()

## My VGG

In [73]:
import torch
import torch.nn as nn
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.max_pool = nn.MaxPool2d(2, 2)
        self.conv_2d_1 = nn.Conv2d(3, 64, 3, 1)
        self.linear_1 = nn.Linear(3968064, 4)
        self.linear_2 = nn.Linear(4, 4)
        self.relu = nn.ReLU()

    def forward(self, x):
        batch_size, h, w, channel = x.shape
        x = x.reshape(batch_size, channel, h, w)

        x = self.conv_2d_1(x)
        x = self.relu(x)
        x = self.max_pool(x)
        x = torch.flatten(x, start_dim=1)
        x = self.linear_1(x)
        x = self.relu(x)
        x = self.linear_2(x)

        return x

## Build YOLO

#### The Design of the YOLO NN was taken from the following paper:

https://arxiv.org/pdf/1506.02640.pdf - "You Only Look Once: Unified, Real-Time Object Detection" by Redmon, Divvala, Girshick, and Farhadi

The following article is YOLO V2:
https://arxiv.org/pdf/1612.08242v1.pdf - "YOLO 9000: Better, Faster, Stronger" by Redmon, and Farhadi

In [74]:
yolo = YOLO()

# Train the Model


- Structured similarly to main.py file from pytorch part of A2

In [81]:
## Define Hyperparameters -- Currently setting values that we can modify
loss_type = "l1"
learning_rate = 0.001
momentum = 0.5
reg = 1e-2

training_batch_size= 10
validation_batch_size= 10

model= "MyModel"

In [82]:
if loss_type == "l1":
    criterion = nn.L1Loss()
    
if loss_type == "l2":
    criterion = nn.MSELoss()
    
if model== "SimpleCNN":
    model= CNN()
elif model == "YOLO":
    model = yolo
elif model == "VGG":
    model = vgg
elif model == "PreTrainedVGG":
    model = preTrainedVGG
elif model == "MyModel":
    model = MyModel()
    
train_ds = WaldoDataset(X_train['new_path'],X_train['new_bb'] ,y_train)
valid_ds = WaldoDataset(X_val['new_path'],X_val['new_bb'],y_val)

train_dl = DataLoader(train_ds, batch_size=training_batch_size, shuffle=True)
valid_dl = DataLoader(valid_ds, batch_size=validation_batch_size)

optimizer = torch.optim.SGD(model.parameters(), learning_rate,
                            momentum=momentum,
                            weight_decay=reg)

In [83]:
training_loop(model= model, optimizer = optimizer, train_dl= train_dl, valid_dl=valid_dl, epochs= 20, criterion= criterion, verbose= True, return_loss= False)

 
--------------------------------------------------------
Training Loss for Epoch 0: 667.1224060058594
Validation Loss for Epoch 0: 286.2743835449219
 
--------------------------------------------------------
Training Loss for Epoch 1: 507.8591995239258
Validation Loss for Epoch 1: 287.76715087890625
 
--------------------------------------------------------
Training Loss for Epoch 2: 494.0905303955078
Validation Loss for Epoch 2: 251.3552703857422


KeyboardInterrupt: 

In [ ]:
loss_type= ["l1"]
learning_rate= [0.001,0.0001]
momentum = [0.9]
reg = [0.01]
batch_size= [10]

all_training_loss, all_validation_loss= hp_grid_search(model_type= "SimpleCNN", 
               lr_list=learning_rate, 
               momentum_list=momentum, 
               reg_list=reg, 
               batch_size_list=batch_size,
               train_ds= train_ds,
               valid_ds= valid_ds,
               optimizer= optimizer, 
               loss_type_list=loss_type,
               epochs= 10,
               save_all_plots="Yes", 
               save_final_plot="Yes",
               final_plot_prefix="Test", 
               return_all_loss= True)